In [1]:
import spacy
import json
from spacy import displacy
import pandas as pd
from tqdm import tqdm
from nltk import word_tokenize

In [2]:
with open('hotel_sentiment_sentences.json', 'r', encoding='utf-8') as f:
    hotel_sentiment_sentences = f.read()
    hotel_sentiment_sentences = json.loads(hotel_sentiment_sentences)

In [3]:
nlp = spacy.load("en_core_web_sm")

In [65]:
spacy_extracted_keywords = {}

In [66]:
for key in tqdm(hotel_sentiment_sentences.keys()):
    sents_pos = hotel_sentiment_sentences[key]['pos']
    sents_neg = hotel_sentiment_sentences[key]['neg']
    hotel_keywords = {'pos': [],
                      'neg': []}
    for sents, tag in [(sents_pos, 'pos'), (sents_neg, 'neg')]:
        keywords = []
        for sent in sents:
            doc = nlp(sent)
            be_tokens = []
            for token in doc:
                if str(token.lemma_) == 'be':
                    be_tokens.append(token)
            for token in be_tokens:
                left = list(token.lefts)
                right = list(token.rights)
                if (len(left) != 0)&(len(right) != 0):
                    left = left[0]
                    right = right[0]
                else:
                    continue
                if (left.pos_ == 'NOUN')&(right.pos_ == 'ADJ'):
                    keyword = str(right) + ' ' + str(left).lower()
                    keywords.append(keyword)
        hotel_keywords[tag] = keywords
    spacy_extracted_keywords[key] = hotel_keywords

100%|██████████████████████████████████████████████████████████████████████████████| 2197/2197 [07:25<00:00,  4.93it/s]


In [67]:
#with open('non-NP_extracted_keywords.json', 'w', encoding='utf-8') as f:
#    f.write(json.dumps(spacy_extracted_keywords))